<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Вспомогательные-функции" data-toc-modified-id="Вспомогательные-функции-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Вспомогательные функции</a></span></li><li><span><a href="#Запуск-обработки" data-toc-modified-id="Запуск-обработки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Запуск обработки</a></span></li><li><span><a href="#Сгенерированная--таблица" data-toc-modified-id="Сгенерированная--таблица-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Сгенерированная  таблица</a></span></li></ul></div>

In [1]:
# импорт библиотек
import sys
import os
import cv2
import re
import string
import copy
import datetime
import pandas as pd
import numpy as np
import Levenshtein as lev
from PIL import Image as PILImage
import easyocr
os.environ["OMP_NUM_THREADS"] = '1'
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from IPython.display import display_html

### Вспомогательные функции

In [2]:
def get_limit_value(stroke):
    # удаление пробелов
    stroke_joined = "".join(stroke['value'].split())
    # поиск даты по маске
    date = None
    mask = r"(.*?)((?:\d{2})(?:\.|-)(?:\d{2})(?:\.|-)(?:\d{2,4}))(.*)"
    try:
        parsed = re.match(mask, stroke_joined)        
        date = parsed.group(2)
        return stroke['y_lu']
    except:
        pass
    return None       

In [3]:
def rename_clasters(claster, name_dict):
    return name_dict[claster]

In [4]:
    # функция оценки адекватности распознавания
    def check_adequacy(stroke, col_type=0):
        # оценка количества
        if col_type == 'quantity':
            try:
                # проверка нахождения количества сданной крови в пределах от 50 до 700 мл
                if 100 <= int(stroke) <= 700:
                    # дополнительная коррекция нормы 450 мл
                    try:
                        re.match(r'(4(?:6|9)(?:0|9|8))', stroke).group(0)
                        stroke = 450
                    except:
                        pass
                    return stroke
                else:
                    return "UNKNOWN"
            except:
                return "UNKNOWN"
        # оценка даты
        elif col_type == 'date':
            # дата приказа о форме справки 
            init_date = datetime.datetime.strptime(r'31.03.2005', '%d.%m.%Y')
            # текущее время
            current_date = datetime.datetime.now()
            date_stroke = ''
            # перебор возможных вариантов записи
            for pattern in [r'%d.%m.%Y', r'%d.%m.%y', r'%d-%m-%Y', r'%d-%m-%y']:
                try:
                    date_stroke = datetime.datetime.strptime(stroke, pattern)
                    break
                except:
                    pass
            # проверка нахождения дат в пределах от даты приказа до текущей
            if date_stroke != '' and (init_date <= date_stroke <= current_date):
                # вывод унифицированного варианта записи напр. 31.12.2020
                return date_stroke.strftime('%d.%m.%Y')
            else:
                return "UNKNOWN"
        # коррекция записи для типа донорства 
        elif col_type == 'type':
            try:                
                # перевод символов в нижний регистр
                stroke_improved = stroke.lower()
                stroke_improved = re.sub('кр.{1}д', 'кр/д', stroke_improved)
                # замена цифр подходящими символами
                stroke_improved = re.sub('3|8|9', 'в', stroke_improved)
                stroke_improved = re.sub('5|6', 'б', stroke_improved)
                # восстановление неполной строки
                if stroke in ['(бв)', 'кр/д']:
                    return r'кр/д (бв)'
                # очистка лишних знаков
                finded = re.match(r'.*((?:кр|пл|ц)(?:/д)|(?:т/ф))(.*)((?:\(бв\))|(?:\(пл\)))', stroke_improved)
                # проверка частей записи по отдельности
                blood_part = finded.group(1)
                reward_type = finded.group(3)
                if blood_part not in ['кр/д', 'пл/д', 'ц/д', 'т/ф']:
                    # коррекция значений с помощью расстояния  Левенштайна
                    if len(blood_part) == 3:
                        if lev.distance(blood_part, 'кр/д') < 2:
                            blood_part = 'кр/д'
                        if lev.distance(blood_part, 'пл/д') < 2:
                            blood_part = 'пл/д'
                    if len(blood_part) == 2:
                        if lev.distance(blood_part, 'т/ф') < 2:
                            blood_part = 'т/ф'
                        if lev.distance(bood_part, 'ц/д') < 2:
                            blood_part = 'ц/д'
                return finded.group(1) + ' ' + finded.group(3)
            except:
                return "UNKNOWN"
        # вывод без проверки    
        else:
            return stroke

### Запуск обработки

In [5]:
puth = r'405/213950.jpg'
img = cv2.imread(puth)

# обрезка верхней части изображения
img = img[int(img.shape[0]/2):img.shape[0], :]
# распознавание без разметки таблицы
reader = easyocr.Reader(lang_list=['ru'])
result = reader.readtext(image=img)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [11]:
data_dict = {'x_lu':[],'y_lu':[],'x_ru':[],'y_ru':[],'x_rb':[],'y_rb':[],'x_lb':[],'y_lb':[],'value':[],'prob':[]}
for item in result:
    # отсев сомнительных данных (с дробными координатами)
    if type(item[0][0][0]) == np.int32:
        # координаты бокса с текстом (не ячейки!)
        data_dict['x_lu'].append(item[0][0][0])
        data_dict['y_lu'].append(item[0][0][1])
        data_dict['x_ru'].append(item[0][1][0])
        data_dict['y_ru'].append(item[0][1][1])
        data_dict['x_rb'].append(item[0][2][0])
        data_dict['y_rb'].append(item[0][2][1])
        data_dict['x_lb'].append(item[0][3][0])
        data_dict['y_lb'].append(item[0][3][1])
        # значение и верятность распознавания
        data_dict['value'].append(item[1])
        data_dict['prob'].append(item[2])
df = pd.DataFrame(data_dict)
# ширина и высота бокса с текстом
df['width'] = df['x_ru'] - df['x_lu']
df['heigth'] = df['y_lb'] - df['y_lu']
# центр бокса с текстом
df['center_horizontal'] = (df['x_ru'] + df['x_lu'])/2
df['center_vertical'] = (df['y_lb'] + df['y_lu'])/2

In [12]:
# поиск верхней и нижней строки с датой
h_min = max(df['y_lu'])
h_max = min(df['y_lu'])
for i in range(len(df)):
    h_curr = get_limit_value(df.iloc[i,:])
    if h_curr != None:
        if h_curr < h_min:
            h_min = h_curr
        elif h_curr > h_max:
            h_max = h_curr
# отсечение по высоте с учетом поправки
df = df[((df['y_lu'] >= h_min - df['heigth'].min()) & (df['y_lu'] <= h_max + df['heigth'].min()))]

In [13]:
X = df['center_horizontal'].values.reshape(-1, 1)
silhouette_max = 0
n_clusters = 9
# подбор числа кластеров
for n in [3,4,6,8,9]:
    clusterer = KMeans(n_clusters=n, n_init="auto", random_state=42)
    cluster_labels = clusterer.fit_predict(X)
    # оценка качества кластеризации
    silhouette_avg = silhouette_score(X, cluster_labels)    
    # оптимальное число кластеров
    if silhouette_avg > silhouette_max:
        silhouette_max = silhouette_avg
        n_clusters = n
# разбивка данных кластеры (столбцы)
clusterer = KMeans(n_clusters=n_clusters, n_init="auto", random_state=42)
cluster_labels = clusterer.fit_predict(X)
df['cluster_labels'] = cluster_labels

In [14]:
# получение образцов кластеров по порядку
claster_samples = []
for j in range(n_clusters):
    for i in df.index:
        if df.loc[i, 'cluster_labels'] == j:
            claster_samples.append(df.loc[i, 'center_horizontal'])
            break

claster_idx = list(np.argsort(claster_samples))
clasters_dict = {claster_name:idx for idx, claster_name  in enumerate(claster_idx)}
df['claster_sorted'] = df['cluster_labels'].apply(lambda x: rename_clasters(x, clasters_dict))

In [15]:
df_final = pd.DataFrame()
for i in range(n_clusters):
    df_claster = df[df['claster_sorted'] == i].sort_values(by=['center_vertical'])
    df_final[i] = df_claster['value'].reset_index(drop=True)

In [16]:
for i in range(df_final.shape[1]):
    if i%3 == 0:
        df_final[i] = df_final[i].apply(lambda x: "".join(x.split()))
        df_final[i] = df_final[i].apply(lambda x: check_adequacy(x, col_type='date'))
    if i%3 == 1:
        df_final[i] = df_final[i].apply(lambda x: check_adequacy(x, col_type='type'))
    if i%3 == 2:
        df_final[i] = df_final[i].apply(lambda x: "".join(x.split()))
        df_final[i] = df_final[i].apply(lambda x: check_adequacy(x, col_type='quantity'))

### Сгенерированная  таблица

In [17]:
df_final

,0,1,2,3,4,5
0,14.04.2014,кр/д (бв),450,21.04.2020,кр/д (бв),450
1,30.05.2018,кр/д (бв),450,27.07.2020,кр/д (бв),350
2,30.07.2018,кр/д (бв),450,19.11.2020,кр/д (бв),450
3,05.10.2018,кр/д (бв),450,22.01.2021,кр/д (бв),450
4,05.12.2018,кр/д (бв),450,15.09.2021,кр/д (бв),450
5,06.02.2019,кр/д (бв),450,18.11.2021,кр/д (бв),450
6,11.04.2019,кр/д (бв),450,02.02.2022,кр/д (бв),450
7,05.07.2019,кр/д (бв),450,04.04.2022,кр/д (бв),450
8,10.09.2019,кр/д (бв),450,21.07.2022,кр/д (бв),450
9,22.11.2019,кр/д (бв),450,04.10.2022,кр/д (бв),450
